In [1]:
import pandas as pd

df = pd.read_csv('tokenized_data.csv', encoding='utf-8')

from sklearn.model_selection import train_test_split

# Split your dataset into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Display the shapes of the training and testing sets
print("Training set shape:", train_df.shape)
print("Testing set shape:", test_df.shape)

Training set shape: (2119, 2)
Testing set shape: (530, 2)


In [2]:
hindi_sentences = df['Hindi_Tokens']
telugu_sentences = df['Telugu_Tokens']

# Split the data into train and validation sets
hindi_train, hindi_val, telugu_train, telugu_val = train_test_split(
    hindi_sentences, telugu_sentences, test_size=0.2, random_state=42)

# Now you have the following variables:
# - hindi_train: List of Hindi sentences for training
# - hindi_val: List of Hindi sentences for validation
# - telugu_train: List of corresponding Telugu sentences for training
# - telugu_val: List of corresponding Telugu sentences for validation

In [3]:
print("Hindi Training set shape:", hindi_train.shape)
print("Hindi Val set shape:", hindi_val.shape)
print("Telugu Training set shape:", telugu_train.shape)
print("Telugu Val set shape:", telugu_val.shape)

Hindi Training set shape: (2119,)
Hindi Val set shape: (530,)
Telugu Training set shape: (2119,)
Telugu Val set shape: (530,)


In [6]:
%pip install tensorflow


  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/9e/b8/ed5f794359d05cd0bffb894c6418da87b93016ee17b669d55c45d1bd5d5b/tensorflow-2.13.0-cp311-cp311-win_amd64.whl.metadata
  Using cached tensorflow-2.13.0-cp311-cp311-win_amd64.whl.metadata (2.6 kB)
Using cached tensorflow-2.13.0-cp311-cp311-win_amd64.whl (1.9 kB)


In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

hindi_tokenizer = Tokenizer(oov_token="<OOV>")
hindi_tokenizer.fit_on_texts(hindi_sentences)
hindi_vocab_size = len(hindi_tokenizer.word_index) + 1
hindi_sequences = hindi_tokenizer.texts_to_sequences(hindi_sentences)

# Tokenize Telugu sentences
telugu_tokenizer = Tokenizer(oov_token="<OOV>")
telugu_tokenizer.fit_on_texts(telugu_sentences)
telugu_vocab_size = len(telugu_tokenizer.word_index) + 1
telugu_sequences = telugu_tokenizer.texts_to_sequences(telugu_sentences)

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/9e/b8/ed5f794359d05cd0bffb894c6418da87b93016ee17b669d55c45d1bd5d5b/tensorflow-2.13.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for tensorflow-intel==2.13.0 from https://files.pythonhosted.org/packages/2f/2f/3c84f675931ce3bcbc7e23acbba1e5d7f05ce769adab48322de57a9f5928/tensorflow_intel-2.13.0-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/126.5 kB ? eta -:--:--
     -------------------------------------- 126.5/126.5 kB 7.8 MB/s eta 0:00:00
  Obtaining dependency information for flatbuffers>=23.1.21 from https://files.pythonhosted.org/packages/6f/12/d5c79ee252793ffe845d58a913197bfa02ae9a0b5c9bc3dc4b58d477b9e7/flatbuffers-23.5.26-py2.py3-none-any.whl.metadata
     ---------------------------------------- 0.0/57.5 kB ? eta -:--:--
     ---------------------------------------- 57.5/57.5 kB 3.1 MB/s eta 0:00:00
  Obtaining dependency 

In [8]:
len(telugu_tokenizer.word_index) + 1, len(hindi_tokenizer.word_index) + 1

(13424, 7249)

In [9]:
max_sequence_length = max(len(seq) for seq in hindi_sequences)
hindi_sequences = pad_sequences(hindi_sequences, maxlen=max_sequence_length, padding="post")
telugu_sequences = pad_sequences(telugu_sequences, maxlen=max_sequence_length, padding="post")

In [10]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Input

# Encoder
encoder_inputs = Input(shape=(max_sequence_length,))
encoder_embedding = Embedding(hindi_vocab_size, 256, input_length=max_sequence_length)(encoder_inputs)
encoder_lstm = LSTM(256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(max_sequence_length,))
decoder_embedding = Embedding(telugu_vocab_size, 256, input_length=max_sequence_length)(decoder_inputs)
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(telugu_vocab_size, activation="softmax")
output = decoder_dense(decoder_outputs)

# Create the model
model = Model([encoder_inputs, decoder_inputs], output)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [11]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 190)]                0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 190)]                0         []                            
                                                                                                  
 embedding (Embedding)       (None, 190, 256)             1855744   ['input_1[0][0]']             
                                                                                                  
 embedding_1 (Embedding)     (None, 190, 256)             3436544   ['input_2[0][0]']             
                                                                                              

In [12]:
import numpy as np
import tensorflow as tf

# Example data (replace with your actual data)
telugu_sequences = np.random.randint(0, 2, size=(64,190))  # Replace with your data
telugu_vocab_size = 13424  # Replace with your vocabulary size

# Convert target data to float16
target_data = tf.keras.utils.to_categorical(telugu_sequences, num_classes=telugu_vocab_size, dtype='float32')


In [13]:
# Prepare data for training
target_data = tf.keras.utils.to_categorical(telugu_sequences, num_classes=telugu_vocab_size, dtype='float32')

# Train the model
model.fit([hindi_sequences, telugu_sequences], target_data, epochs=5, batch_size=64, validation_split=0.2)

ValueError: Data cardinality is ambiguous:
  x sizes: 2119, 64
  y sizes: 64
Make sure all arrays contain the same number of samples.